In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error

In [2]:

train_data=pd.read_csv("/home/jupyter/kaggle-notebooks/Dataset/train.csv")
X=train_data.drop(columns=["id","Hardness"])
y=train_data["Hardness"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


## Random Forest Regressor

In [3]:
from sklearn.ensemble import RandomForestRegressor
RF_regressor= RandomForestRegressor()
RF_regressor.fit(X_train,y_train)
y_pred=RF_regressor.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae



0.6810000000000009

## HistGradientBoosting Regressor

In [4]:
from sklearn.ensemble import HistGradientBoostingRegressor
HGBR=HistGradientBoostingRegressor()
HGBR.fit(X_train,y_train)
y_pred=HGBR.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.6381275442355352

## SVM Regressor

In [5]:
from sklearn import svm
svr = svm.SVR()
svr.fit(X_train,y_train)
y_pred=svr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.9019036394186755

## XGB Regressor

In [6]:
!pip install xgboost

In [7]:
import xgboost as xgb
xgbr=xgb.XGBRegressor()
xgbr.fit(X_train,y_train)
y_pred=xgbr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.6781597137451172

## Catboost Regressor

In [8]:
from catboost import CatBoostRegressor
cbr=CatBoostRegressor()
cbr.fit(X_train,y_train)
y_pred=cbr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae


Learning rate set to 0.057227
0:	learn: 1.6577228	total: 50.6ms	remaining: 50.6s
1:	learn: 1.6298259	total: 53.8ms	remaining: 26.9s
2:	learn: 1.6028105	total: 56.8ms	remaining: 18.9s
3:	learn: 1.5778444	total: 59.6ms	remaining: 14.8s
4:	learn: 1.5552126	total: 62.4ms	remaining: 12.4s
5:	learn: 1.5346130	total: 65.9ms	remaining: 10.9s
6:	learn: 1.5151060	total: 69ms	remaining: 9.79s
7:	learn: 1.4969527	total: 72.4ms	remaining: 8.98s
8:	learn: 1.4809247	total: 75.9ms	remaining: 8.36s
9:	learn: 1.4661157	total: 79.1ms	remaining: 7.83s
10:	learn: 1.4519091	total: 81.8ms	remaining: 7.35s
11:	learn: 1.4386087	total: 84.8ms	remaining: 6.98s
12:	learn: 1.4285161	total: 89.1ms	remaining: 6.76s
13:	learn: 1.4174654	total: 92.1ms	remaining: 6.48s
14:	learn: 1.4068424	total: 94.8ms	remaining: 6.22s
15:	learn: 1.3970505	total: 97.5ms	remaining: 6s
16:	learn: 1.3889331	total: 100ms	remaining: 5.8s
17:	learn: 1.3809595	total: 103ms	remaining: 5.62s
18:	learn: 1.3733801	total: 105ms	remaining: 5.45s
1

0.6620364074719269

## LGBM Regressor


In [9]:
from lightgbm import LGBMRegressor
lgbmr=LGBMRegressor()
lgbmr.fit(X_train,y_train)
y_pred=lgbmr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2462
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


0.6508308930496491

## Gradient Boosting Regressor

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
gbr=GradientBoostingRegressor()
gbr.fit(X_train,y_train)
y_pred=gbr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.6967739198178702

## ExtraTree Regressor

In [11]:
from sklearn.tree import ExtraTreeRegressor
etr=ExtraTreeRegressor()
etr.fit(X_train,y_train)
y_pred=etr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.7999999999999998

## Decision Tree Regressor

In [12]:
from sklearn.tree import DecisionTreeRegressor
dcr=DecisionTreeRegressor()
dcr.fit(X_train,y_train)
y_pred=dcr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.7999999999999998

Lets try optuna

In [13]:
!pip install optuna

In [22]:
import optuna
from sklearn.model_selection import cross_val_score
# define objective function for optuna
def objective(trial):
    # Define the hyperparameters to tune
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.002, 0.2,log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1200, step=50),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10,log=True),
        'gamma': trial.suggest_float('gamma', 1e-5, 10,log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    # Train an XGB Regressor model with the given hyperparameters
    model = xgb.XGBRegressor(**params,random_state=42)
    scores=cross_val_score(model,X,y,cv=5)
    return scores.mean()


# Create an Optuna study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score found by Optuna
print(f'Best score: {study.best_value:.4f}')
print(f'Best params: {study.best_params}')

[I 2023-11-16 20:58:38,351] A new study created in memory with name: no-name-40fbd30b-f193-47b3-9968-7de803d013bb
[I 2023-11-16 20:58:39,250] Trial 0 finished with value: 0.46182050035736255 and parameters: {'max_depth': 4, 'learning_rate': 0.030134715385036582, 'subsample': 0.9439100180944738, 'colsample_bytree': 0.6085544742190341, 'n_estimators': 200, 'reg_alpha': 0.0002721576483216536, 'reg_lambda': 0.0038527311118156686, 'gamma': 0.5486824081788256, 'min_child_weight': 7}. Best is trial 0 with value: 0.46182050035736255.
[I 2023-11-16 20:59:07,442] Trial 1 finished with value: 0.4744415631623909 and parameters: {'max_depth': 14, 'learning_rate': 0.0023949721960925133, 'subsample': 0.7447064086152421, 'colsample_bytree': 0.5259027872855522, 'n_estimators': 850, 'reg_alpha': 0.017909910230844133, 'reg_lambda': 0.0024495534938904903, 'gamma': 0.3973700978946879, 'min_child_weight': 9}. Best is trial 0 with value: 0.46182050035736255.
[I 2023-11-16 20:59:34,332] Trial 2 finished with 

Best score: 0.3195
Best params: {'max_depth': 16, 'learning_rate': 0.19287322574787402, 'subsample': 0.5043406338484141, 'colsample_bytree': 0.5973496990770337, 'n_estimators': 1000, 'reg_alpha': 7.452596231177216e-05, 'reg_lambda': 0.00025666795977588255, 'gamma': 0.0007120544050877592, 'min_child_weight': 10}


In [16]:
best_par=study.best_params
model=xgb.XGBRegressor(**best_par)
model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5190840017923333, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=1.6569056353278303e-05,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.007975207804537535,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [17]:
test_dataset=pd.read_csv("/home/jupyter/kaggle-notebooks/Dataset/test.csv")
sample_submission = pd.read_csv('/home/jupyter/kaggle-notebooks/Dataset/sample_submission.csv')

In [18]:
test_dataset["Hardness"]=model.predict(test_dataset.drop(columns='id'))
test_dataset.head()

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,10407,884.0,121.420000,35.36,5.28,82.561240,9.370384,2.298,1.9072,1.200000,0.461899,1.79459,4.085202
1,10408,90.0,9.931960,18.00,5.60,39.568056,12.086300,2.868,1.6520,0.864000,0.476196,1.41194,4.139578
2,10409,116.0,7.767992,11.60,4.80,23.231818,11.023840,2.644,1.7940,0.960000,0.499514,0.78834,4.961176
3,10410,100.0,9.107996,10.00,4.80,20.298893,12.086300,2.828,1.6620,0.792000,0.495796,1.20466,4.725186
4,10411,55.0,4.030000,11.00,4.00,22.977675,11.280950,2.455,1.7500,0.893333,0.476095,0.93456,4.777887


In [19]:
sample_submission=test_dataset[["id","Hardness"]]
sample_submission.to_csv("Mysubmission.csv",index=False)